In [ ]:
import sys
# Important deps
!apt update && apt install -y aria2 python3-pip git libgl1-mesa-glx

# Clone the repo
!git clone https://github.com/kk-digital/kcg-ml-sd1p4.git --recurse-submodules -b lora

%cd kcg-ml-sd1p4

In [ ]:
# Install reqs
!pip3 install -r train/requirements.txt

In [ ]:
# Download model weights
!aria2c https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -o ./input/model/sd-v1-4.ckpt -j 10

In [ ]:
# Let's now run the 'test suite'
!python3 train/lora_train.py